### All required libraries import to support the GUI, dataset operation, and recommendation logic

- **tkinter and ttk**: Used for creating the graphical interface that allows the user to interact with the recommendation system.
https://docs.python.org/3/library/tkinter.html

- **pandas**: Provides powerful tools for reading, manipulating, and analyzing structured data (like movie and ratings data).
https://pandas.pydata.org/docs/user_guide/index.html#user-guide

- **joblib**: Enables saving and loading machine learning models and large data efficiently (used here to load models and data).
https://joblib.readthedocs.io/en/latest/

- **random**: Built-in Python module to perform random operations, such as selecting movies for user input.
https://docs.python.org/3/library/random.html

- **cosine_similarity from sklearn.metrics.pairwise**: Measures how similar two vectors are — useful for comparing movie features like genres or cast.
https://scikit-learn.org/stable/user_guide.html

In [16]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import joblib
import random
from sklearn.metrics.pairwise import cosine_similarity

#### This block loads the pre-trained machine learning models and preprocessed data that are essential for generating movie recommendations.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
https://surprise.readthedocs.io/en/stable/matrix_factorization.html#singular-value-decomposition

In [19]:
movies_df = joblib.load("models/movies_df.pkl").reset_index(drop=True)
genre_tfidf_matrix = joblib.load("models/genre_tfidf_matrix.pkl")
cast_tfidf_matrix = joblib.load("models/cast_tfidf_matrix.pkl")
svd_model = joblib.load("models/svd_model.pkl")

#### Select 10 popular and diverse movies from the dataset for the user to rate initially. This helps personalize the recommendation system based on user input.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html

In [22]:
top_movies = movies_df.sort_values("numVotes", ascending=False).head(1000)
initial_rating_movies = top_movies.sample(n=10, random_state=random.randint(0, 10000)).reset_index(drop=True)

####  1 - Initialize and configure the main application window for the GUI using Tkinter.

https://docs.python.org/3/library/tkinter.html#tkinter.Tk

#### 2 - Creates and places a frame inside the main window to hold the rating-related widgets.

https://docs.python.org/3/library/tkinter.html#tkinter.Frame

#### 3 - Sets up a scrollable area inside the rating_frame to accommodate many movie rating widgets in a clean, scrollable view.

Creates a Canvas widget inside rating_frame. The canvas provides a space where widgets can be drawn or added, enabling scrolling of content.
https://docs.python.org/3/library/tkinter.html#canvas

Adds a vertical scrollbar linked to the canvas’s vertical scrolling (yview). It allows vertical navigation through long content.
https://www.pythontutorial.net/tkinter/tkinter-scrollbar/

Creates a Frame inside the canvas to hold the actual content (movie widgets). This is where movie cards will be displayed, and this frame will be scrolled.
https://www.pythontutorial.net/tkinter/tkinter-frame/

Canvas doc: https://tkdocs.com/tutorial/canvas.html

#### 4 - Makes the scroll functionality responsive to content size changes inside scrollable_frame.

Binds an event that triggers every time the scrollable frame is resized, and updates the scrollable region of the canvas to include everything.
https://blog.teclado.com/tkinter-scrollable-frames/

#### 5 - Connects the scrollable_frame to the canvas and enables vertical scrolling using the scrollbar.

Embeds the scrollable_frame inside the canvas at the top-left corner (0, 0).

Anchor="nw" means the embedded frame is pinned to the northwest (top-left) of the canvas.

canvas.configure(yscrollcommand=scrollbar.set) links the scrollbar to the canvas’s vertical view and ensures the scrollbar updates as the user scrolls through the canvas content.

https://stackoverflow.com/questions/58009398/tkinter-what-is-a-window-when-calling-tk-canvas-create-window
https://www.geeksforgeeks.org/how-to-add-a-scrollbar-to-a-group-of-widgets-in-tkinter/

#### 6 - Arranges the canvas and scrollbar widgets inside the parent rating_frame using the Tkinter layout manager .pack().

Places the canvas on the left side of the parent frame (rating_frame); fill="both": Makes the canvas stretch to fill both width and height of the available space; expand=True: Allows the canvas to grow and shrink when the window resizes.

Places the scrollbar on the right side of the frame; fill="y": Makes the scrollbar stretch vertically.

#### 7 - Dynamically creates a set of movie rating widgets in the UI, allowing users to rate 10 well-known, randomly chosen movies.

rating_vars = [] initializes an empty list to store movie–rating variable pairs for later use in recommendation logic.

for i, row in initial_rating_movies.iterrows(): iterates through each of the 10 selected movies.

frame = ttk.LabelFrame(...) creates a labeled frame for each movie displaying its title and release year.

ttk.Label(...) displays the movie’s genres and cast information inside the frame.

rating_var = tk.DoubleVar(value=5.0) initializes a Tkinter variable to hold the user’s rating, defaulting to 5.0.

rating_dropdown = ttk.Combobox(...) creates a dropdown allowing the user to select a rating from 1 to 10.
https://tkdocs.com/tutorial/widgets.html#combobox

#### 8 - Collects the movies that were rated by the user and stores.

rated_titles: names of the movies rated.

rated_indices: the corresponding index in movies_df for similarity lookup.

We need these to compute similarity scores (TF-IDF) and filter out movies already seen/rated.

https://pandas.pydata.org/docs/user_guide/indexing.html

#### 9 - Compute Hybrid Recommendation Scores.

Skips any already-rated movies when making recommendations.

Compute genre, and cast similarity using tf-idf vectors. Also compute predicted rating using svd collab filtering. Combine all into a weighted hybrid score blending content-based and collab filtering for better recommendation.
https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

#### 10 - Sorts the list of recommendations in descending order based on their hybrid score (which is at index 3 in each tuple), and then selects the top 10.

recommendations is a list of tuples.

key=lambda x: x[3] tells sorted() to sort by the hybrid score.

reverse=True means higher scores appear first (descending order).

[:10] slices the first 10 results from the sorted list.
https://docs.python.org/3/library/functions.html#sorted
https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions

#### 11 - Clears old recommendation widgets from the GUI to avoid stacking new recommendations on top of previous ones.

https://www.geeksforgeeks.org/how-to-clear-out-a-frame-in-the-tkinter/

#### 12 - Display recommendations.

Creates a LabelFrame for each movie and shows title, genres, cast, calc hybrid score, and preditcted rating.
https://docs.python.org/3/library/tkinter.ttk.html

#### 13 - Creates a button.

root is the parent window.

text="Get Recommendations" sets the label on the button. 

command=recommend binds the button click event to execute the recommend() function.

In [37]:
#1
root = tk.Tk()
root.title("Movie Recommender")
root.geometry("800x600")

#2
rating_frame = tk.Frame(root)
rating_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

#3
canvas = tk.Canvas(rating_frame)
scrollbar = ttk.Scrollbar(rating_frame, orient="vertical", command=canvas.yview)
scrollable_frame = ttk.Frame(canvas)

#4
scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

#5
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

#6
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

#7
rating_vars = []

for i, row in initial_rating_movies.iterrows():
    frame = ttk.LabelFrame(scrollable_frame, text=f"{row['primaryTitle']} ({row['startYear']})")
    frame.pack(fill="x", pady=5)

    ttk.Label(frame, text=f"Genres: {row['genres']}").pack(anchor="w")
    ttk.Label(frame, text=f"Cast: {row['cast_crew']}").pack(anchor="w")

    rating_var = tk.DoubleVar(value=5.0)
    rating_dropdown = ttk.Combobox(frame, textvariable=rating_var, values=[i for i in range(1, 11)], width=5)
    rating_dropdown.pack(anchor="w")
    rating_vars.append((row, rating_var))

#8
def recommend():
    rated_titles = []
    rated_indices = []
    for movie, rating_var in rating_vars:
        match = movies_df[movies_df["primaryTitle"] == movie["primaryTitle"]]
        if not match.empty:
            idx = match.index[0]
            rated_indices.append(idx)
            rated_titles.append(movie["primaryTitle"])

    #9
    recommendations = []
    for i in range(len(movies_df)):
        if movies_df.iloc[i]["primaryTitle"] in rated_titles:
            continue
        genre_sim = cosine_similarity(genre_tfidf_matrix[i], genre_tfidf_matrix[rated_indices]).mean()
        cast_sim = cosine_similarity(cast_tfidf_matrix[i], cast_tfidf_matrix[rated_indices]).mean()
        pred_rating = svd_model.predict(9999, movies_df.iloc[i]["primaryTitle"]).est
        score = 0.5 * pred_rating + 0.25 * genre_sim + 0.25 * cast_sim
        recommendations.append((movies_df.iloc[i]["primaryTitle"], movies_df.iloc[i]["genres"], 
                                 movies_df.iloc[i]["cast_crew"], score, pred_rating))

    #10
    recommendations = sorted(recommendations, key=lambda x: x[3], reverse=True)[:10]

    #11
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    #12
    for title, genres, cast, score, pred_rating in recommendations:
        rec_frame = ttk.LabelFrame(scrollable_frame, text=f"{title}")
        rec_frame.pack(fill="x", pady=5)
        ttk.Label(rec_frame, text=f"Genres: {genres}").pack(anchor="w")
        ttk.Label(rec_frame, text=f"Cast: {cast}").pack(anchor="w")
        ttk.Label(rec_frame, text=f"Predicted Rating: {score:.2f}").pack(anchor="w")
        ttk.Label(rec_frame, text=f"Score: {pred_rating:.2f}").pack(anchor="w")

#13
recommend_btn = tk.Button(root, text="Get Recommendations", command=recommend)
recommend_btn.pack(pady=10)

root.mainloop()

2025-05-16 16:47:34.901 python[85947:19092895] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-16 16:47:34.901 python[85947:19092895] +[IMKInputSession subclass]: chose IMKInputSession_Modern
